# Определение тональности текста

__Задача:__ обучить модель (бинарной классификации) распознавать тональность текста (негативная и позитивная).

Набор данных взят с сайта https://study.mokoron.com/ и представляет собой набор из 114911 положительных, 111923 отрицательных записей из русскоязычного твиттера.

### Описание столбцов

– id: уникальный номер сообщения в системе twitter;

– tdate: дата публикации сообщения (твита);

– tmane: имя пользователя, опубликовавшего сообщение;

– ttext: текст сообщения (твита);

– ttype: класс, к которому принадлежит текст (положительный, отрицательный, нейтральный);

– tret: количество ретвитов (количество копирований этого сообщения другими пользователями);

– trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;

– tfav: число сколько раз данное сообщение было добавлено в избранное другими пользователями;

– tstcount: число всех сообщений пользователя в сети twitter;

– tfol: количество фоловеров пользователя (тех людей, которые читают пользователя);

– tfrien: количество друзей пользователя (те люди, которых читает пользователь);

– listcount: количество листов-подписок в которые добавлен твиттер-пользователь.

In [2]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as ts
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, Activation, Flatten, Dense, Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from keras.utils import to_categorical

C:\Users\Anna.muravyeva\Anaconda3\envs\NewEnvPython37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Anna.muravyeva\Anaconda3\envs\NewEnvPython37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Anna.muravyeva\Anaconda3\envs\NewEnvPython37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Anna.murav

### Загрузка данных/ создание датасета

In [3]:
colnames=['id', 'tdate', 'tmane', 'ttext', 'ttype', 'tret', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount']
positive_df = pd.read_csv('positive.csv', sep=';', names=colnames, header=None, error_bad_lines=False)
negative_df = pd.read_csv('negative.csv', sep=';', names=colnames, header=None, error_bad_lines=False)

In [4]:
negative_df

,id,tdate,tmane,ttext,ttype,tret,trep,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0
...,...,...,...,...,...,...,...,...,...,...,...,...
111918,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
111919,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
111920,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
111921,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [5]:
positive_df.ttext

0         @first_timee хоть я и школота, но поверь, у на...
1         Да, все-таки он немного похож на него. Но мой ...
2         RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3         RT @digger2912: "Кто то в углу сидит и погибае...
4         @irina_dyshkant Вот что значит страшилка :D\nН...
                                ...                        
114906    Спала в родительском доме, на своей кровати......
114907    RT @jebesilofyt: Эх... Мы немного решили сокра...
114908    Что происходит со мной, когда в эфире #proacti...
114909    "Любимая,я подарю тебе эту звезду..." Имя како...
114910    @Ma_che_rie посмотри #непытайтесьпокинутьомск ...
Name: ttext, Length: 114911, dtype: object

In [6]:
Full_df = positive_df.append(negative_df)
Full_df = Full_df.reset_index(drop=True)

### Проверка данных

In [7]:
Full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         226834 non-null  int64 
 1   tdate      226834 non-null  int64 
 2   tmane      226834 non-null  object
 3   ttext      226834 non-null  object
 4   ttype      226834 non-null  int64 
 5   tret       226834 non-null  int64 
 6   trep       226834 non-null  int64 
 7   tfav       226834 non-null  int64 
 8   tstcount   226834 non-null  int64 
 9   tfol       226834 non-null  int64 
 10  tfrien     226834 non-null  int64 
 11  listcount  226834 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 20.8+ MB


Нулевых значений нет. Дата представлена в неясном виде, однако, предварительно можно сказать, что для задачи классификации текста она не нужна

In [8]:
Full_df.describe()

,id,tdate,ttype,tret,trep,tfav,tstcount,tfol,tfrien,listcount
count,2.268340e+05,2.268340e+05,226834.000000,226834.0,226834.000000,226834.000000,2.268340e+05,2.268340e+05,226834.000000,226834.000000
mean,4.132836e+17,1.387369e+09,0.013173,0.0,3.118249,0.001243,7.584972e+03,6.770083e+02,332.682746,10.125475
std,4.535456e+15,1.081337e+06,0.999915,0.0,105.898784,0.035611,2.039972e+04,9.764450e+03,1915.268263,112.100088
min,4.089067e+17,1.386326e+09,-1.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,4.100277e+17,1.386593e+09,-1.000000,0.0,0.000000,0.000000,4.730000e+02,2.700000e+01,29.000000,0.000000
50%,4.110529e+17,1.386838e+09,1.000000,0.0,0.000000,0.000000,2.026000e+03,8.300000e+01,72.000000,0.000000
75%,4.157585e+17,1.387960e+09,1.000000,0.0,0.000000,0.000000,7.562000e+03,2.570000e+02,197.000000,2.000000
max,4.251386e+17,1.390196e+09,1.000000,0.0,13817.000000,2.000000,1.138639e+06,1.582807e+06,388311.000000,16915.000000


In [9]:
Full_df.ttype.value_counts()

 1    114911
-1    111923
Name: ttype, dtype: int64

Значения классов соответсвуют ожиданиям, нет серьезного дисбаланса.

### Подготовка данных

In [10]:
tokenizer = Tokenizer(num_words=None, char_level=True, oov_token='UNK')

In [11]:
train_dataframe = []

In [12]:
for row in positive_df['ttext']:
    tokenizer.fit_on_texts(row.lower())
    train_dataframe.append((row.lower(), 1))
    
for row in negative_df['ttext']:
    tokenizer.fit_on_texts(row.lower())
    train_dataframe.append((row.lower(), 0))

In [13]:
#train_dataframe

In [14]:
len(train_dataframe)

226834

### Подготовка модели

In [15]:
train_data, test_data = train_test_split(train_dataframe, test_size=0.1)
train_data, val_data = train_test_split(train_data, test_size=0.2)

In [16]:
len(val_data)

40830

In [17]:
alphabet = "абвгдеёжзиклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

In [18]:
char_dict.values()

dict_values([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100])

In [19]:
tokenizer.word_index = char_dict.copy()
tokenizer.word_index[tokenizer.oov_token] = max(char_dict.values()) + 1 # 101

In [20]:
train_df = pd.DataFrame.from_records(train_data)
val_df = pd.DataFrame.from_records(val_data)
test_df = pd.DataFrame.from_records(test_data)

In [21]:
test_df

,0,1
0,@randa_rin @lovemytommi на компе я к сожалению...,0
1,"@annamichelekam а как по мне, скучно. ушло вдо...",0
2,поиграла блин с кисой:( http://t.co/ulik1gzi6v,0
3,зима а у меня бабочка летает по комнате ))) ht...,1
4,"rt @wolagasivelu: танчики обновились. патч, сц...",0
...,...,...
22679,rt @yulia76stimban5: ненавижу болеть( сидеть д...,0
22680,"вот будет классно, если мега пати у нашей пара...",0
22681,"rt @xyhycecixyt: да, на работе у меня тоже yuk...",1
22682,rt @ediny_marsy: @s_o_o_o_o_n *о* великолепный...,0


In [22]:
train_df

,0,1
0,rt @mypokunoned: я верю в дружбу между мужчино...,1
1,"хочу встретить человека из своего сна ,но я ег...",0
2,"всегда мечтал,чтоб на мой день рождения выпал ...",1
3,#appearancesbsf #interviewbsf / / все интервь...,1
4,"@sashaiyulskaya хаха, я с радостью отвечу на л...",1
...,...,...
163315,@nprivaty она ж еще после егэ расклеилась полн...,0
163316,"@ars_rnd @enki_rnd ох, сегодня вряд ли, меня д...",0
163317,обидно когда ты даешь водителю в маршрутке 5.5...,0
163318,"ого, рой джонс заявил, что хотел бы получить р...",0


In [23]:
train_texts = train_df[0].values
train_texts = [s for s in train_texts]

test_texts = test_df[0].values
test_texts = [s for s in test_texts]

val_texts = val_df[0].values
val_texts = [s for s in val_texts]

In [24]:
#train_texts

In [25]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

In [26]:
#train_sequences

In [27]:
train_data = pad_sequences(train_sequences, maxlen=50, padding = 'post')
test_data = pad_sequences(test_sequences, maxlen=50, padding = 'post')
val_data = pad_sequences(val_sequences, maxlen=50, padding='post')

In [28]:
train_data

array([[ 28, 101,  18, ...,  38,  62,  49],
       [  6,  19,  10, ...,  24,  20,  95],
       [  3,  15,  19, ...,  74,  91,  96],
       ...,
       [ 11,   6, 101, ...,  15,  70,  95],
       [ 10,   3,  28, ..., 101,  74,  95],
       [101,   9,   1, ...,   6,  14,  29]])

In [29]:
train_data = np.array(train_data)
test_data = np.array(test_data)
val_data = np.array(val_data)

In [30]:
train_data

array([[ 28, 101,  18, ...,  38,  62,  49],
       [  6,  19,  10, ...,  24,  20,  95],
       [  3,  15,  19, ...,  74,  91,  96],
       ...,
       [ 11,   6, 101, ...,  15,  70,  95],
       [ 10,   3,  28, ..., 101,  74,  95],
       [101,   9,   1, ...,   6,  14,  29]])

In [31]:
train_classes = train_df[1].values
train_class_list = [x for x in train_classes]

test_classes = test_df[1].values
test_class_list = [x for x in test_classes]

val_classes = val_df[1].values
val_class_list = [x for x in val_classes]

In [32]:
test_class_list

[0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,


In [33]:
train_classes

array([1, 0, 1, ..., 0, 0, 1], dtype=int64)

In [34]:
#train_class_list

In [35]:
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)
val_classes = to_categorical(val_class_list)

In [36]:
to_categorical(val_class_list)

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

### Параметры модели

In [37]:
vocab_size = len(tokenizer.word_index)

In [38]:
embedding_weights = []
embedding_weights.append(np.zeros(vocab_size))

In [39]:
embedding_weights

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]

In [40]:
for char, i in tokenizer.word_index.items():
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [41]:
embedding_weights

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [43]:
input_size = 50
vocab_size = 101
embedding_size = 101
conv_layers = [[256, 7, 3], # 256, 7, 3
              ]
fully_connected_layers = [384, 384]
num_of_classes = 2
dropout_p = 0.7
optimizer = 'adam'
loss = 'binary_crossentropy'

In [44]:
loss

'binary_crossentropy'

In [45]:
embedding_layer = Embedding(vocab_size+1,
                           embedding_size,
                           input_length=input_size,
                           weights=[embedding_weights])

In [46]:
inputs = Input(shape=(input_size,), name='input', dtype='int64')

x = embedding_layer(inputs)

for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)
x = Flatten()(x)

for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x)
    x = Dropout(dropout_p)(x)

predictions = Dense(num_of_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 101)           10302     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 44, 256)           181248    
_________________________________________________________________
activation_1 (Activation)    (None, 44, 256)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 14, 256)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3584)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 384)               137

In [47]:
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]

x_test = test_data
y_test = test_classes

indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]

x_test = test_data
y_test = test_classes

x_val = val_data
y_val = val_classes

In [48]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [49]:
total_weights = len(train_dataframe)

positive_weight = len(positive_df) / total_weights
negative_weight = len(negative_df) / total_weights

class_weights = [positive_weight, negative_weight]

In [50]:
class_weights

[0.5065863142209721, 0.49341368577902783]

In [51]:
y_val

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

### Обучение модели

In [52]:
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          #class_weight=class_weights,
          batch_size=50,
          epochs=10,
          verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 163320 samples, validate on 40830 samples
Epoch 1/10
 - 144s - loss: 0.3448 - accuracy: 0.7966 - val_loss: 0.3216 - val_accuracy: 0.8171
Epoch 2/10
 - 145s - loss: 0.3118 - accuracy: 0.8263 - val_loss: 0.3115 - val_accuracy: 0.8253
Epoch 3/10
 - 145s - loss: 0.3015 - accuracy: 0.8355 - val_loss: 0.3105 - val_accuracy: 0.8280
Epoch 4/10
 - 145s - loss: 0.2944 - accuracy: 0.8422 - val_loss: 0.3104 - val_accuracy: 0.8296
Epoch 5/10
 - 145s - loss: 0.2850 - accuracy: 0.8490 - val_loss: 0.3127 - val_accuracy: 0.8309
Epoch 6/10
 - 145s - loss: 0.2764 - accuracy: 0.8567 - val_loss: 0.3183 - val_accuracy: 0.8306
Epoch 7/10
 - 149s - loss: 0.2688 - accuracy: 0.8621 - val_loss: 0.3136 - val_accuracy: 0.8286
Epoch 8/10
 - 156s - loss: 0.2601 - accuracy: 0.8675 - val_loss: 0.3210 - val_accuracy: 0.8284
Epoch 9/10
 - 149s - loss: 0.2500 - accuracy: 0.8746 - val_loss: 0.3409 - val_accuracy: 0.8309

In [53]:
print(model.evaluate(x_test, y_test))
model.metrics_names

22684/22684 [==============================] - 4s 178us/step
[0.32982874726980077, 0.8326132893562317]


['loss', 'accuracy']

In [54]:
def predict(text):

    text = text.lower()
    #print(text)
    res = []
    for sentence in text.replace('\n', '. ').split('. '):

        tokenized_text = []
        tokenized_text.append(sentence)

        tokenized_text = tokenizer.texts_to_sequences(tokenized_text)
        tokenized_text = pad_sequences(tokenized_text, maxlen=50, padding = 'post')
        
        #print(model.predict(tokenized_text))
        prediction = np.argmax(model.predict(tokenized_text))
        if prediction == 0:
            pred = 'negativ'
        elif prediction == 1:
            pred = 'positiv'
       
        res.append((sentence, pred))

    return res

### Проверка

In [62]:
text = 'я всю ночь пила кофе, смотрела сериалы, прекрасно провела время'
predict(text)

[('я всю ночь пила кофе, смотрела сериалы, прекрасно провела время',
  'positiv')]

In [63]:
text = 'Добрый день! Я бы хотел купить автомобиль, зелененький'
predict(text)

[('добрый день! я бы хотел купить автомобиль, зелененький', 'positiv')]

In [64]:
text = 'Я бы хотел вернуть автомобиль, он сломался'
predict(text)

[('я бы хотел вернуть автомобиль, он сломался', 'negativ')]

In [66]:
text = 'Я очень недоволен обслуживанием, вы плохо выполнили свою работу!'
predict(text)

[('я очень недоволен обслуживанием, вы плохо выполнили свою работу!',
  'negativ')]